In [3]:
# pdf에서 페이지 숫자 현황을 paddleOCR을 사용하여 결과를 json으로 산출하는 프로그램


import os # 디렉토리 접근
import json # json 자료구조 사용
import numpy as np # 이미지를 numpy 배열로 변환
from PIL import Image
from pdf2image import convert_from_path # poppler 설치 필수!
from paddleocr import PaddleOCR # paddleocr == 3.1.0 # paddlepaddle == 3.1.0
import re





# pdf to images 함수
# pdf의 경로와 이미지로 변환할 해상도를 변수로 받는다. 기본 dpi는 300으로 고정
def convert_pdf_to_images(pdf_path, dpi = 300):

    # 라이브러리 함수 사용 from pdf2image import convert_from_path
    # Poppler 설치 필수
    # pdf를 이미지로 변환하는 함수
    images = convert_from_path(pdf_path, dpi = dpi)

    
    # 이미지 산출 경로 받기
    output_dir = os.path.splitext(pdf_path)[0]
    os.makedirs(output_dir, exist_ok = True)

    
    # pdf index length 계산을 위한 이미지 저장 배열
    image_paths = []

    
    # pdf의 있는 모든 페이지를 열거(enumerate) 하여 경로에 이미지로 저장
    for i, image in enumerate(images):
        
        img_path = os.path.join(output_dir, f"{os.path.basename(output_dir)}_{i}.png")
        
        image.save(img_path, "PNG")
        
        image_paths.append(img_path)

    
    # 열겨된 이미지 경로 배열 리턴
    return image_paths






# 파이썬 자료구조를 json으로 변환하는 함수
def convert_for_json(obj):

    # 파이썬 튜플 구조일 경우
    if isinstance(obj, (list, tuple)):
        return [convert_for_json(o) for o in obj]

    # 파이썬 딕셔너리 구조일 경우
    elif isinstance(obj, dict): 
        return {k: convert_for_json(v) for k, v in obj.items()}

    # 원시 타입일 경우
    elif isinstance(obj, (int, float, str)):
        return obj

    # 그외 아무것도 아니면 String 객체로 반환
    else:
        return str(obj)




# 이미지 하단 일정 높이만 추출
def crop_bottom_region(image, height):
    
    x1 = 0
    x2 = image.width
    y2 = image.height
    y1 = y2 - height
    
    if y1 < 0:
        raise ValueError("입력한 높이가 전체 이미지보다 큽니다")

    # 글자 인식을 위한 사각형 구간 좌표 설정
    crop_box = (x1, y1, x2, y2)
    
    cropped = image.crop(crop_box)

    
    # crop_box와 numpy자료형 2개 반환
    return np.array(cropped), crop_box







# 문자열 좌표 추출 이전 코드
#def extract_coords(poly_str):
#    coords = re.findall(r"\[(\d+)\s+(\d+)\]", poly_str)
#    
#    return [(int(x), int(y)) for x, y in coords]


# 문자열 좌표 추출
def extract_coords(poly_str):
    coords = re.findall(r"\[(\d+)\s+(\d+)\]", poly_str)
    coords = [(int(x), int(y)) for x, y in coords]
    if len(coords) == 2:
        coords = two_points_to_box(coords)
    return coords


# 좌표 4개로 보정
def two_points_to_box(coords):
    (x1, y1), (x2, y2) = coords
    return [(x1, y1), (x2, y1), (x2, y2), (x1, y2)]



# 좌표 평균 x값
def get_center_x(coords):
    xs = [pt[0] for pt in coords]
    
    return np.mean(xs)


# 사각형 구간 픽셀 좌표를 원본 이미지의 전체 픽셀 기준으로 보정하는 함수
def adjust_coords_to_original(coords, crop_box):
    x_offset = crop_box[0]
    y_offset = crop_box[1]
    
    return [(x + x_offset, y + y_offset) for (x, y) in coords]









# 페이지 숫자의 left right 위치 탐지
def add_position_to_ocr_result(ocr_result, image_width, crop_box):
    center_x = image_width / 2
    updated_result = []

    
    for res in ocr_result:
        dt_polys = res.get("dt_polys", [])
        rec_texts = res.get("rec_texts", [])
        positions = []

        
        for poly_str in dt_polys:
            if not poly_str or poly_str == "[]":
                positions.append("unknown")
                continue


            # 리팩토링 필요 부분
            coords = extract_coords(poly_str)
            
            coords = adjust_coords_to_original(coords, crop_box)  # 좌표 보정

            #print(f"좌표 개수: {len(coords)}")

            
            cx = get_center_x(coords)
            pos = "left" if cx < center_x else "right"
            positions.append(pos)

        updated_res = []
        
        for idx, text in enumerate(rec_texts):


            # 리팩토링 필요 부분 
            coords = extract_coords(dt_polys[idx]) if idx < len(dt_polys) else []

            
            coords = adjust_coords_to_original(coords, crop_box)  # 좌표 보정

            
            position = positions[idx] if idx < len(positions) else "unknown"
            updated_res.append({
                "text": text,
                "position": position,
                "coords": coords
            })

        new_res = dict(res)
        new_res["rec_texts_with_position"] = updated_res
        updated_result.append(new_res)

    return updated_result





# 숫자 텍스트 여부 확인 0 ~ 9999 범위
def is_valid_number(text):
    if not text.strip().isdigit():
        return False
    num = int(text.strip())
    return 0 <= num <= 9999



    
# 이미지 파일 넘버 정렬
def natural_keys(text):
    return [int(s) if s.isdigit() else s.lower() for s in re.split(r'(\d+)', text)]


    


# 메인 함수
# main entry
if __name__ == "__main__":

    # numpy 배열 생략 해제
    #np.set_printoptions(threshold=np.inf)
    np.set_printoptions(threshold=784,linewidth=np.inf)


    
    # 디버깅할려면 이 3개 코드 주석처리하세요
    pdf_path = input("pdf 경로를 입력: ").strip()
    image_paths = convert_pdf_to_images(pdf_path) #  dpi기본값 300
    print(f"\n{len(image_paths)}장 이미지 변환 완료\n")

    # 원래 코드
    image_dir = os.path.dirname(image_paths[0])

    # 디버깅용 - 이미지 변환 시간 스킵
    #image_dir = "/home/zen31/Desktop/paddlework/data/2/2/3"
    #image_dir = "/home/zen31/Desktop/paddlework/data/2/2"

    
    # 페이지 숫자 부분 인식할 사각형 구간 설정
    # 이미지 맨 아래서 부터 사각형 높이 설정
    height = int(input("인식할 픽셀 높이를 입력: ").strip())


    # 페이지 숫자만 인식하는대 한국어로 설정할 필요가 있는지??? 영어 인식률이 훨 높지않나 - 해본결과 한국어가 더 인식률높음 바꿀필요 x
    ocr = PaddleOCR(lang='korean')

    # 인식할 이미지 파일 유효 포멧 형식
    valid_exts = [".png", ".jpg", ".jpeg"]
    image_files = [f for f in os.listdir(image_dir) if os.path.splitext(f)[1].lower() in valid_exts]

    # 이미지 파일이 존재하지 않는경우
    if not image_files:
        print("해당 디렉토리에 이미지가 존재하지 않음")
        exit(1)

    
    # 이미지 파일명 인덱스의 오름차순으로 정렬
    image_files.sort(key=natural_keys)

    
    all_results = []

    # 이미지 파일 전체 ocr인식 시작
    for filename in image_files:

        # pdf 이미지 저장한 경로를 주입?
        image_path = os.path.join(image_dir, filename)

        # try catch statements
        try:
            image = Image.open(image_path)
            cropped_np, crop_box = crop_bottom_region(image, height)
            image_width = crop_box[2] - crop_box[0]


            
            #result = ocr.predict(cropped_np)

            # paddle ocr 버젼 충돌시 아래 코드 사용
            # ocr.ocr()은 deprecated 예정 ocr.predict() 사용 권장
            result = ocr.ocr(cropped_np)

            # json으로 저장
            converted_result = convert_for_json(result)

            
            # 페이지숫자 ocr인식시 0 ~ 9999 사이만 인식
            for res in converted_result:
                texts = res.get("rec_texts", [])
                filtered = [t for t in texts if is_valid_number(t)]
                res["rec_texts"] = filtered

            # 원래 코드
            #result_with_pos = add_position_to_ocr_result(converted_result, image_width)


            # 여기
            # 페이지 숫자 사각형 인식 구간 좌표를 전체 이미지 좌표로 보정
            result_with_pos = add_position_to_ocr_result(converted_result, image_width, crop_box)


            
            empty_text_flag = True

            
            for res in converted_result:
                rec_texts = res.get("rec_texts", [])
                if any(text.strip() for text in rec_texts):
                    empty_text_flag = False
                    break

            # 결과 json으로 저장
            result_json = {
                "filename": filename,
                "ocr_result": result_with_pos,
                "empty_text": empty_text_flag
            }

            
            all_results.append(result_json)
            print(f"OCR 인식 완료: {filename}")


        # 페이지 숫자 이미지 인식 오류 예외처리
        except Exception as e:
            print(f"{filename} 예외 발생: {e}")


    
    output_path = os.path.join(image_dir, "pdf_page_ocr_results.json")
    
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)


    
    print(f"\n결과 json으로 저장 완료: {output_path}")
    



pdf 경로를 입력:  /home/zen31/Desktop/paddlework/data/2/2.pdf



70장 이미지 변환 완료



인식할 픽셀 높이를 입력:  500


Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Using official model (PP-LCNet_x1_0_doc_ori), the model files will be automatically downloaded and saved in /home/zen31/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

...

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Using official model (korean_PP-OCRv5_mobile_rec), the model files will be automatically downloaded and saved in /home/zen31/.paddlex/official_models.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

/tmp/ipykernel_27103/904012978.py:288: DeprecationWarning: Please use `predict` instead.
  result = ocr.ocr(cropped_np)


OCR 인식 완료: 2_0.png
OCR 인식 완료: 2_1.png
OCR 인식 완료: 2_2.png
OCR 인식 완료: 2_3.png
OCR 인식 완료: 2_4.png
OCR 인식 완료: 2_5.png
OCR 인식 완료: 2_6.png
OCR 인식 완료: 2_7.png
OCR 인식 완료: 2_8.png
OCR 인식 완료: 2_9.png
OCR 인식 완료: 2_10.png
OCR 인식 완료: 2_11.png
OCR 인식 완료: 2_12.png
OCR 인식 완료: 2_13.png
OCR 인식 완료: 2_14.png
OCR 인식 완료: 2_15.png
OCR 인식 완료: 2_16.png
OCR 인식 완료: 2_17.png
OCR 인식 완료: 2_18.png
OCR 인식 완료: 2_19.png
OCR 인식 완료: 2_20.png
OCR 인식 완료: 2_21.png
OCR 인식 완료: 2_22.png
OCR 인식 완료: 2_23.png
OCR 인식 완료: 2_24.png
OCR 인식 완료: 2_25.png
OCR 인식 완료: 2_26.png
OCR 인식 완료: 2_27.png
OCR 인식 완료: 2_28.png
OCR 인식 완료: 2_29.png
OCR 인식 완료: 2_30.png
OCR 인식 완료: 2_31.png
OCR 인식 완료: 2_32.png
OCR 인식 완료: 2_33.png
OCR 인식 완료: 2_34.png
OCR 인식 완료: 2_35.png
OCR 인식 완료: 2_36.png
OCR 인식 완료: 2_37.png
OCR 인식 완료: 2_38.png
OCR 인식 완료: 2_39.png
OCR 인식 완료: 2_40.png
OCR 인식 완료: 2_41.png
OCR 인식 완료: 2_42.png
OCR 인식 완료: 2_43.png
OCR 인식 완료: 2_44.png
OCR 인식 완료: 2_45.png
OCR 인식 완료: 2_46.png
OCR 인식 완료: 2_47.png
OCR 인식 완료: 2_48.png
OCR 인식 완료: 2_49.png
OCR 인식 완료:

In [5]:
# ocr로 페이지 숫자 결과물 json파일 분석 프로그램

import json
import os



# 페이지 인덱스 번호 추출
def extract_page_number(fname):
    try:
        base = os.path.basename(fname)
        name, _ = os.path.splitext(base)
        return int(name.split("_")[1]) # page index int 형으로 반환
    except:
        return None






def analyze_pdf_page_patterns():
    
    json_path = input("json 결과 경로 입력: ").strip()

    
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)



    
    # 첫 페이지부터 끝까지 left right 검사 
    alternating_results = []
    
    base_position = None
    
    expected = None
    
    base_filename = None

    # 최초로 페이지 숫자가 나온 이미지가 기준점이 된다.
    found_base = False
    
    alternating_wrong_files = set()


    
    # data = json.load(f)
    # data는 pdf ocr인식 결과 json 파일 입니다.
    for item in data:

        # pdf ocr 인식결과 json파일을 열어서 함께 보시면 이해가 빠릅니다.
        
        # pdf ocr 인식결과 json 애트리뷰트 filename -> 이미지 파일
        filename = item["filename"]

        # pdf ocr 인식결과 json 애트리뷰트 empty_text -> 글자 인식 true false 결과값 가져오기
        empty = item.get("empty_text", True)


        # 첫번째 경우의 수
        # found_base가 false이고 empty가 false인 경우
        # 즉 기준점(base)이 될 최초의 페이지숫자가 나온 이미지가 없고, 글자도 인식 못한경우 -> 페이지 숫자가 없는 페이지들 넘기기
        if not found_base and not empty:

            # pdf ocr 인식결과 json 애트리뷰트 ocr_result -> 이 애트리뷰트에 이미지당 ocr인식 결과들이 저장되어있음.
            rec_infos = item.get("ocr_result", [])

            
            # 페이지숫자 왼쪽 오른쪽 정보
            positions = []


            # 첫번째 이미지에 위치정보와 페이지 숫자가 있으면 현 이미지가 base 이미지가 된다. 만일없으면 현재 코드 스킵
            for block in rec_infos:
                for res in block.get("rec_texts_with_position", []):
                    pos = res.get("position", "unknown")
                    if pos in ["left", "right"]:
                        positions.append(pos)

            
            # 페이지 숫자를 인식하였으나, 왼쪽 오른쪽 페이지 숫자 위치 정보가 없을경우
            if not positions:
                alternating_results.append((filename, "위치정보없음", "left right 위치정보가 없어 base가 될 수 없습니다"))
                continue # 아래 left right 카운트 코드 실행 방지


            
            # 첫번째 이미지에 위치정보와 페이지 숫자가 있으면 현 이미지가 base 이미지가 된다.
            left_count = positions.count("left")
            right_count = positions.count("right")
            base_position = "left" if left_count >= right_count else "right"
            expected = "left" if base_position == "right" else "right"
            base_filename = filename

            
            # 현재 이미지를 base로 판정
            found_base = True
            alternating_results.append((filename, base_position, "base"))
            continue # 다음 상태로 건너뛰기 아래코드들 스킵



        

        # 두번째 경우의 수
        # 기준점이미지는 없고, empty가 true인경우 즉 아무 숫자도 인식안된 이미지 인 경우 스킵
        if not found_base:
            alternating_results.append((filename, "empty", "base 페이지 이전까지 스킵"))
            continue

        current_position = None



        

        # 세번째 경우의 수 
        # 기준점 이미지를 찾고, 이미지에 페이지 숫자가 있는경우
        # 가장 일반적인 경우
        if not empty:
            rec_infos = item.get("ocr_result", [])
            positions = []

            for block in rec_infos:
                for res in block.get("rec_texts_with_position", []):
                    pos = res.get("position", "unknown")
                    if pos in ["left", "right"]:
                        positions.append(pos)

            if not positions:
                alternating_results.append((filename, "no position", f"expected {expected}"))
                expected = "right" if expected == "left" else "left"
                continue

            left_count = positions.count("left")
            right_count = positions.count("right")
            current_position = "left" if left_count >= right_count else "right"

            if current_position == expected:
                alternating_results.append((filename, current_position, "OK"))
            else:
                alternating_results.append((filename, current_position, f"방향 오류 (올바른 방향 {expected})"))
                alternating_wrong_files.add(filename)




        
        # 네번째 경우의 수
        # 기준점 이미지를 찾고, 이미지에 페이지 숫자가 없는경우 ( pdf에 해당 단락이 끝나 페이지 숫자가 없는경우 같은것들) 
        else:
            alternating_results.append((filename, "empty", f"넘기는 페이지, 예상 패턴 {expected}"))


        
        expected = "right" if expected == "left" else "left"




    


    
    
    # 생략되거나(건너뛴) 중복된 페이지 숫자 확인
    number_sequence = []

    for item in data:
        filename = item["filename"]
        ocr_result = item.get("ocr_result", [])
        if not ocr_result:
            continue
        texts_with_pos = ocr_result[0].get("rec_texts_with_position", [])
        for obj in texts_with_pos:
            text = obj.get("text", "")
            if text.isdigit():
                number_sequence.append((int(text), filename))

    number_sequence.sort()
    skipped_files = []

    for i in range(1, len(number_sequence)):
        current_num, current_file = number_sequence[i]
        prev_num, _ = number_sequence[i - 1]
        if current_num != prev_num + 1:
            skipped_files.append(current_file)



    
    
    # 중복된 position left right 방향 탐지
    prev_position = None
    violations = []

    for item in data:
        filename = item["filename"]
        ocr_result = item.get("ocr_result", [])
        if not ocr_result:
            continue
        texts_with_pos = ocr_result[0].get("rec_texts_with_position", [])
        if not texts_with_pos:
            continue
        current_position = texts_with_pos[0].get("position", "").lower()
        if prev_position is not None and current_position == prev_position:
            violations.append({
                "filename": filename,
                "position": current_position
            })
        prev_position = current_position





    
    
    # 탐지된 파일 저장

    # 건너뛰거나 중복되거나 이상한 숫자가 탐지된 페이지
    skipped_set = set(skipped_files)

    # 중복된 left right 페이지
    violation_set = set(v["filename"] for v in violations)

    # left/right 패턴이 어긋난 페이지
    alternating_set = set(alternating_wrong_files)

    
    problem_set = skipped_set | violation_set | alternating_set

    result_data = []

    # 에러코드 raw 저장
    for item in data:
        filename = item["filename"]
        if filename not in problem_set:
            continue

        ocr_result = item.get("ocr_result", [])
        if not ocr_result:
            continue

        texts_with_pos = ocr_result[0].get("rec_texts_with_position", [])
        code = []
        if filename in skipped_set:
            code.append(1)
        if filename in violation_set:
            code.append(2)
        if filename in alternating_set:
            code.append(3)

        error_code = code[0] if len(code) == 1 else code

        output = {
            "filename": filename,
            "rec_texts_with_position": texts_with_pos,
            "error_code": error_code
        }

        result_data.append(output)






    # 모든 탐지 상황 저장
    all_info = {
        "base_filename": base_filename,
        "base_position": base_position,
        "left_right_pattern": [
            {"filename": fname, "position": pos, "status": status}
            for fname, pos, status in alternating_results
        ],
        "skipped_files": skipped_files,
        "violations": violations
    }






    
    
    # 에러 코드 contents 요약
    errors = []

    # 해당 내용이 존재할 경우에만 errors 객체에 데이터 추가

    # 건너뛰거나 또는 좌우 방향이 틀린경우
    skipped_and_violation = skipped_set | violation_set
    
    if skipped_and_violation:
        errors.append({
        "code": "1_2",
        "description": "페이지 숫자가 건너뛰거나, 중복, 이상한숫자가 탐지 & left/right 방향이 동시에 발생한 pdf 인덱스 번호입니다.",

        # 2_3.png에서 인덱스 번호 3만 추출, 추출후 오름차순 정렬
        "filename": sorted([extract_page_number(f) for f in skipped_and_violation if extract_page_number(f) is not None])
    })

    
    # 이전 코드
    #if skipped_set:
    #    errors.append({
    #        "code": 1,
    #        "description": "건너뛰거나 중복되거나 이상한 숫자가 탐지된 페이지입니다",
    #        "filename": list(skipped_set)
    #    })

    #if violation_set:
    #    errors.append({
    #        "code": 2,
    #        "description": "중복된 left right가 존재하는 뒷페이지 입니다. 앞 페이지와 함께 확인해주세요",
    #        "filename": list(violation_set)
    #    })

    
    if alternating_set:
        errors.append({
            "code": 3,
            "description": "left와 right 패턴이 어긋난 페이지입니다 앞 페이지와 함계 확인해주세요",
            "filename": list(alternating_set)
        })




    

        
    
    # 최종 json 출력
    output_dir = os.path.dirname(json_path)
    output_path = os.path.join(output_dir, "pdf_result.json")

    final_output = {
        "error_results": result_data,
        "all_info": all_info,
        "errors": errors
    }

    # json 덤프
    with open(output_path, "w", encoding = "utf-8") as f:
        json.dump(final_output, f, ensure_ascii = False, indent = 2)







    
    # 콘솔 출력
    print("\n탐지된 비정상 파일")
    for entry in result_data:
        print(json.dumps(entry, ensure_ascii=False, indent=2))

    print("\nleft right 탐지 정보")
    print(f"기준(base) 페이지: {base_filename} → '{base_position}' 시작")
    print("\n")
    for fname, pos, status in alternating_results:
        print(f"{fname:20} | {pos:8} | {status}")

    print("\n건너뛴 숫자가 있는 파일들")
    for fname in skipped_files:
        print(f"  - {fname}")

    print("\n중복된 left, right")
    for v in violations:
        print(f"  - {v['filename']} 에서 '{v['position']}' 중복 발생")

    print(f"\n탐지 결과가 '{output_path}'에 저장")










# main entry
if __name__ == "__main__":
    analyze_pdf_page_patterns()


json 결과 경로 입력:  /home/zen31/Desktop/paddlework/data/2/2/pdf_page_ocr_results.json



탐지된 비정상 파일
{
  "filename": "2_34.png",
  "rec_texts_with_position": [
    {
      "text": "9114",
      "position": "right",
      "coords": [
        [
          2156,
          3009
        ],
        [
          2156,
          3009
        ],
        [
          2156,
          3021
        ],
        [
          2156,
          3021
        ]
      ]
    },
    {
      "text": "27",
      "position": "right",
      "coords": [
        [
          2307,
          3120
        ],
        [
          2307,
          3120
        ],
        [
          2307,
          3165
        ],
        [
          2307,
          3165
        ]
      ]
    }
  ],
  "error_code": 1
}
{
  "filename": "2_36.png",
  "rec_texts_with_position": [
    {
      "text": "007",
      "position": "right",
      "coords": [
        [
          2131,
          3009
        ],
        [
          2131,
          3009
        ],
        [
          2131,
          3020
        ],
        [
          2131,
    